In [2]:
# Initial setup
import os
os.environ["POLARS_UNKNOWN_EXTENSION_TYPE_BEHAVIOR"] = "load_as_storage"
import polars as pl
from dotenv import load_dotenv
load_dotenv()
db_eng = "adbc"
db_uri = os.getenv("DB_URI")

In [7]:
# Querying characteristics for each IP address and storing the result locally
characteristic_query = """
SELECT
    ip_addr,
    AVG(ping_rttavg) as mean_rtt,
    STDDEV(ping_rttavg) as jitter,
    AVG(ping_ploss) as mean_loss,
    COUNT(*) as observation_count
FROM public.ping
WHERE ping_ploss < 100
    AND ping_date >= '2009-01-01'
GROUP BY ip_addr
HAVING COUNT(*) > 50
"""

df_characteristic = pl.read_database_uri(query=characteristic_query, uri=db_uri, engine=db_eng)
os.makedirs("data", exist_ok=True)
df_characteristic.write_parquet("data/characteristics.parquet")

In [3]:
# Clustering IP addresses by their characteristics
%load_ext cuml.accel
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
df = pl.read_parquet("data/characteristics.parquet")

features = ["mean_rtt", "jitter", "mean_loss"]
X = df.select(features).to_numpy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(X_scaled)

df = df.with_columns(pl.Series(name="cluster", values=clusters))

df.write_parquet("data/characteristics_clustered.parquet")

The cuml.accel extension is already loaded. To reload it, use:
  %reload_ext cuml.accel
